# India AQI Prediction System - Exploratory Data Analysis

## Overview
This notebook performs comprehensive exploratory data analysis on air quality data for India to understand patterns, trends, and relationships that will inform our prediction models.

### Objectives:
1. **Data Quality Assessment** - Missing values, outliers, data distribution
2. **Temporal Analysis** - Time series patterns, seasonality, trends
3. **Spatial Analysis** - Geographic distribution of pollution levels
4. **Correlation Analysis** - Relationships between pollutants and weather
5. **Feature Engineering Insights** - Identify important features for modeling

### Data Sources:
- Historical AQI data from major Indian cities
- Weather data (temperature, humidity, wind, pressure)
- Factory locations and emissions data
- Forest cover data

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import folium
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Libraries imported successfully!")
print(f"Analysis performed on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Data Loading and Initial Exploration

In [ ]:
# Create realistic sample data for analysis
np.random.seed(42)

# Generate sample AQI data for major Indian cities
cities = {
    'Delhi': {'lat': 28.6139, 'lon': 77.2090, 'base_aqi': 180},
    'Mumbai': {'lat': 19.0760, 'lon': 72.8777, 'base_aqi': 120},
    'Bangalore': {'lat': 12.9716, 'lon': 77.5946, 'base_aqi': 85},
    'Chennai': {'lat': 13.0827, 'lon': 80.2707, 'base_aqi': 95},
    'Kolkata': {'lat': 22.5726, 'lon': 88.3639, 'base_aqi': 160},
    'Hyderabad': {'lat': 17.3850, 'lon': 78.4867, 'base_aqi': 110},
    'Pune': {'lat': 18.5204, 'lon': 73.8567, 'base_aqi': 100},
    'Ahmedabad': {'lat': 23.0225, 'lon': 72.5714, 'base_aqi': 130}
}

# Generate 2 years of hourly data
start_date = datetime(2023, 1, 1)
end_date = datetime(2024, 12, 31)
date_range = pd.date_range(start_date, end_date, freq='H')

data = []

for timestamp in date_range:
    for city, info in cities.items():
        # Seasonal factors
        month = timestamp.month
        if month in [11, 12, 1]:  # Winter
            seasonal_factor = 1.4
        elif month in [4, 5]:  # Summer
            seasonal_factor = 1.2
        elif month in [6, 7, 8, 9]:  # Monsoon
            seasonal_factor = 0.7
        else:
            seasonal_factor = 1.0
        
        # Hourly factors (rush hours effect)
        hour = timestamp.hour
        if hour in [7, 8, 9, 18, 19, 20]:  # Rush hours
            hourly_factor = 1.3
        elif hour in [10, 11, 14, 15]:  # Good dispersion
            hourly_factor = 0.8
        else:
            hourly_factor = 1.0
        
        # Base AQI with variations
        base_aqi = info['base_aqi']
        aqi = base_aqi * seasonal_factor * hourly_factor + np.random.normal(0, 15)
        aqi = max(10, min(500, aqi))
        
        # Generate related pollutants
        pm25 = aqi * 0.4 + np.random.normal(0, 5)
        pm10 = aqi * 0.6 + np.random.normal(0, 8)
        no2 = 20 + (aqi / 10) + np.random.normal(0, 5)
        o3 = 60 + (aqi / 8) + np.random.normal(0, 10)
        so2 = 10 + (aqi / 15) + np.random.normal(0, 3)
        co = 1 + (aqi / 100) + np.random.normal(0, 0.3)
        
        # Weather data
        temp_base = 25 + (info['lat'] - 20) * -0.5 + 10 * np.cos(2 * np.pi * timestamp.dayofyear / 365)
        temperature = temp_base + np.random.normal(0, 3)
        humidity = 60 + 20 * np.sin(2 * np.pi * timestamp.dayofyear / 365) + np.random.normal(0, 10)
        wind_speed = 3 + np.random.exponential(2)
        pressure = 1013 + np.random.normal(0, 5)
        
        data.append({
            'timestamp': timestamp,
            'city': city,
            'latitude': info['lat'],
            'longitude': info['lon'],
            'aqi': aqi,
            'pm25': max(0, pm25),
            'pm10': max(0, pm10),
            'no2': max(0, no2),
            'o3': max(0, o3),
            'so2': max(0, so2),
            'co': max(0, co),
            'temperature': temperature,
            'humidity': max(0, min(100, humidity)),
            'wind_speed': max(0, wind_speed),
            'pressure': pressure,
            'month': month,
            'hour': hour,
            'day_of_week': timestamp.dayofweek,
            'is_weekend': timestamp.dayofweek >= 5
        })

# Create DataFrame
df = pd.DataFrame(data)

print(f"Dataset created with {len(df):,} records")
print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
print(f"Cities: {df['city'].nunique()} - {list(df['city'].unique())}")
print(f"\nDataset shape: {df.shape}")
df.head()

## 2. Data Quality Assessment

In [ ]:
# Basic dataset information
print("=== DATASET OVERVIEW ===")
print(df.info())
print("\n=== MISSING VALUES ===")
missing_values = df.isnull().sum()
missing_percentage = (missing_values / len(df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_values,
    'Missing Percentage': missing_percentage
})
print(missing_df[missing_df['Missing Count'] > 0])

print("\n=== BASIC STATISTICS ===")
print(df.describe())

In [ ]:
# Missing values heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(df.isnull(), cbar=True, cmap='viridis', yticklabels=False)
plt.title('Missing Values Heatmap', fontsize=16, fontweight='bold')
plt.xlabel('Features')
plt.ylabel('Records')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('plots/missing_values_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Missing values heatmap saved to plots/missing_values_heatmap.png")

## 3. AQI Distribution Analysis

In [ ]:
# AQI distribution by city
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('AQI Distribution Analysis', fontsize=16, fontweight='bold')

# Overall AQI distribution
axes[0, 0].hist(df['aqi'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Overall AQI Distribution')
axes[0, 0].set_xlabel('AQI')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['aqi'].mean(), color='red', linestyle='--', label=f'Mean: {df["aqi"].mean():.1f}')
axes[0, 0].legend()

# AQI by city (box plot)
df.boxplot(column='aqi', by='city', ax=axes[0, 1])
axes[0, 1].set_title('AQI Distribution by City')
axes[0, 1].set_xlabel('City')
axes[0, 1].set_ylabel('AQI')
plt.setp(axes[0, 1].xaxis.get_majorticklabels(), rotation=45)

# AQI categories distribution
def get_aqi_category(aqi):
    if aqi <= 50:
        return 'Good'
    elif aqi <= 100:
        return 'Moderate'
    elif aqi <= 150:
        return 'Unhealthy for Sensitive'
    elif aqi <= 200:
        return 'Unhealthy'
    elif aqi <= 300:
        return 'Very Unhealthy'
    else:
        return 'Hazardous'

df['aqi_category'] = df['aqi'].apply(get_aqi_category)
category_counts = df['aqi_category'].value_counts()

axes[1, 0].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
axes[1, 0].set_title('AQI Categories Distribution')

# Top polluted cities (average AQI)
city_avg_aqi = df.groupby('city')['aqi'].mean().sort_values(ascending=False)
axes[1, 1].bar(city_avg_aqi.index, city_avg_aqi.values, color='coral')
axes[1, 1].set_title('Average AQI by City')
axes[1, 1].set_xlabel('City')
axes[1, 1].set_ylabel('Average AQI')
plt.setp(axes[1, 1].xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.savefig('plots/aqi_distribution_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ AQI distribution analysis saved to plots/aqi_distribution_analysis.png")

## 4. Temporal Patterns Analysis

In [ ]:
# Time series analysis
fig, axes = plt.subplots(3, 2, figsize=(18, 15))
fig.suptitle('Temporal Patterns in Air Quality', fontsize=16, fontweight='bold')

# Daily average AQI trend (sample period)
daily_aqi = df.groupby(df['timestamp'].dt.date)['aqi'].mean()
sample_period = daily_aqi.iloc[:90]  # First 90 days
axes[0, 0].plot(sample_period.index, sample_period.values, linewidth=1, alpha=0.7)
axes[0, 0].set_title('Daily AQI Trend (First 90 Days)')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Average AQI')
axes[0, 0].tick_params(axis='x', rotation=45)

# Monthly patterns
monthly_aqi = df.groupby('month')['aqi'].mean()
axes[0, 1].bar(monthly_aqi.index, monthly_aqi.values, color='lightcoral')
axes[0, 1].set_title('Average AQI by Month')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Average AQI')
axes[0, 1].set_xticks(range(1, 13))

# Hourly patterns
hourly_aqi = df.groupby('hour')['aqi'].mean()
axes[1, 0].plot(hourly_aqi.index, hourly_aqi.values, marker='o', linewidth=2, color='green')
axes[1, 0].set_title('Average AQI by Hour of Day')
axes[1, 0].set_xlabel('Hour')
axes[1, 0].set_ylabel('Average AQI')
axes[1, 0].set_xticks(range(0, 24, 2))
axes[1, 0].grid(True, alpha=0.3)

# Day of week patterns
dow_aqi = df.groupby('day_of_week')['aqi'].mean()
dow_labels = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
axes[1, 1].bar(range(7), dow_aqi.values, color='purple', alpha=0.7)
axes[1, 1].set_title('Average AQI by Day of Week')
axes[1, 1].set_xlabel('Day of Week')
axes[1, 1].set_ylabel('Average AQI')
axes[1, 1].set_xticks(range(7))
axes[1, 1].set_xticklabels(dow_labels)

# Weekend vs Weekday comparison
weekend_comparison = df.groupby('is_weekend')['aqi'].mean()
axes[2, 0].bar(['Weekday', 'Weekend'], weekend_comparison.values, color=['blue', 'orange'])
axes[2, 0].set_title('Weekday vs Weekend AQI')
axes[2, 0].set_ylabel('Average AQI')

# Seasonal comparison
def get_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8, 9]:
        return 'Monsoon'
    else:
        return 'Post-Monsoon'

df['season'] = df['month'].apply(get_season)
seasonal_aqi = df.groupby('season')['aqi'].mean()
axes[2, 1].bar(seasonal_aqi.index, seasonal_aqi.values, color=['brown', 'green', 'blue', 'orange'])
axes[2, 1].set_title('Average AQI by Season')
axes[2, 1].set_ylabel('Average AQI')
plt.setp(axes[2, 1].xaxis.get_majorticklabels(), rotation=45)

plt.tight_layout()
plt.savefig('plots/temporal_patterns.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Temporal patterns analysis saved to plots/temporal_patterns.png")

## 5. Correlation Analysis

In [ ]:
# Correlation matrix
correlation_features = ['aqi', 'pm25', 'pm10', 'no2', 'o3', 'so2', 'co', 
                       'temperature', 'humidity', 'wind_speed', 'pressure']

correlation_matrix = df[correlation_features].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='RdYlBu_r', center=0,
            square=True, linewidths=0.5, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Correlation Matrix: Pollutants and Weather Variables', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('plots/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Correlation matrix saved to plots/correlation_matrix.png")

# Strong correlations summary
print("\n=== STRONG CORRELATIONS WITH AQI ===")
aqi_correlations = correlation_matrix['aqi'].abs().sort_values(ascending=False)
print(aqi_correlations[aqi_correlations > 0.5])

In [ ]:
# Scatter plots for key relationships
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Key Relationships with AQI', fontsize=16, fontweight='bold')

# AQI vs PM2.5
axes[0, 0].scatter(df['pm25'], df['aqi'], alpha=0.5, color='red')
axes[0, 0].set_xlabel('PM2.5 (μg/m³)')
axes[0, 0].set_ylabel('AQI')
axes[0, 0].set_title('AQI vs PM2.5')

# AQI vs Temperature
axes[0, 1].scatter(df['temperature'], df['aqi'], alpha=0.5, color='orange')
axes[0, 1].set_xlabel('Temperature (°C)')
axes[0, 1].set_ylabel('AQI')
axes[0, 1].set_title('AQI vs Temperature')

# AQI vs Wind Speed
axes[0, 2].scatter(df['wind_speed'], df['aqi'], alpha=0.5, color='green')
axes[0, 2].set_xlabel('Wind Speed (m/s)')
axes[0, 2].set_ylabel('AQI')
axes[0, 2].set_title('AQI vs Wind Speed')

# AQI vs Humidity
axes[1, 0].scatter(df['humidity'], df['aqi'], alpha=0.5, color='blue')
axes[1, 0].set_xlabel('Humidity (%)')
axes[1, 0].set_ylabel('AQI')
axes[1, 0].set_title('AQI vs Humidity')

# PM2.5 vs PM10
axes[1, 1].scatter(df['pm25'], df['pm10'], alpha=0.5, color='purple')
axes[1, 1].set_xlabel('PM2.5 (μg/m³)')
axes[1, 1].set_ylabel('PM10 (μg/m³)')
axes[1, 1].set_title('PM2.5 vs PM10')

# NO2 vs AQI
axes[1, 2].scatter(df['no2'], df['aqi'], alpha=0.5, color='brown')
axes[1, 2].set_xlabel('NO2 (μg/m³)')
axes[1, 2].set_ylabel('AQI')
axes[1, 2].set_title('AQI vs NO2')

plt.tight_layout()
plt.savefig('plots/scatter_relationships.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Scatter relationships saved to plots/scatter_relationships.png")

## 6. Spatial Distribution Analysis

In [ ]:
# City-wise AQI comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Spatial Distribution of Air Quality', fontsize=16, fontweight='bold')

# Average AQI by city
city_stats = df.groupby('city').agg({
    'aqi': ['mean', 'std', 'min', 'max'],
    'pm25': 'mean',
    'pm10': 'mean'
}).round(2)

city_avg_aqi = df.groupby('city')['aqi'].mean().sort_values(ascending=True)
axes[0, 0].barh(city_avg_aqi.index, city_avg_aqi.values, color='lightblue')
axes[0, 0].set_title('Average AQI by City')
axes[0, 0].set_xlabel('Average AQI')

# AQI variability by city
city_std_aqi = df.groupby('city')['aqi'].std().sort_values(ascending=True)
axes[0, 1].barh(city_std_aqi.index, city_std_aqi.values, color='lightcoral')
axes[0, 1].set_title('AQI Variability by City (Std Dev)')
axes[0, 1].set_xlabel('AQI Standard Deviation')

# PM2.5 vs PM10 by city
city_pm = df.groupby('city')[['pm25', 'pm10']].mean()
axes[1, 0].scatter(city_pm['pm25'], city_pm['pm10'], s=100, alpha=0.7)
for i, city in enumerate(city_pm.index):
    axes[1, 0].annotate(city, (city_pm['pm25'].iloc[i], city_pm['pm10'].iloc[i]), 
                       xytext=(5, 5), textcoords='offset points', fontsize=8)
axes[1, 0].set_xlabel('Average PM2.5')
axes[1, 0].set_ylabel('Average PM10')
axes[1, 0].set_title('PM2.5 vs PM10 by City')

# Worst air quality days by city
worst_days = df[df['aqi'] > 200].groupby('city').size().sort_values(ascending=True)
if len(worst_days) > 0:
    axes[1, 1].barh(worst_days.index, worst_days.values, color='red', alpha=0.7)
    axes[1, 1].set_title('Days with AQI > 200 by City')
    axes[1, 1].set_xlabel('Number of Days')
else:
    axes[1, 1].text(0.5, 0.5, 'No days with AQI > 200', ha='center', va='center', transform=axes[1, 1].transAxes)
    axes[1, 1].set_title('Days with AQI > 200 by City')

plt.tight_layout()
plt.savefig('plots/spatial_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n=== CITY STATISTICS ===")
print(city_stats)
print("\n✅ Spatial distribution analysis saved to plots/spatial_distribution.png")

In [ ]:
# Create interactive map with city AQI levels
city_summary = df.groupby(['city', 'latitude', 'longitude']).agg({
    'aqi': ['mean', 'max'],
    'pm25': 'mean',
    'pm10': 'mean'
}).round(2)

city_summary.columns = ['avg_aqi', 'max_aqi', 'avg_pm25', 'avg_pm10']
city_summary = city_summary.reset_index()

# Create folium map
m = folium.Map(location=[20.5937, 78.9629], zoom_start=5)

for _, row in city_summary.iterrows():
    # Color based on AQI level
    if row['avg_aqi'] <= 50:
        color = 'green'
    elif row['avg_aqi'] <= 100:
        color = 'yellow'
    elif row['avg_aqi'] <= 150:
        color = 'orange'
    elif row['avg_aqi'] <= 200:
        color = 'red'
    else:
        color = 'purple'
    
    popup_text = f"""
    <b>{row['city']}</b><br>
    Avg AQI: {row['avg_aqi']:.1f}<br>
    Max AQI: {row['max_aqi']:.1f}<br>
    Avg PM2.5: {row['avg_pm25']:.1f} μg/m³<br>
    Avg PM10: {row['avg_pm10']:.1f} μg/m³
    """
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=10,
        popup=popup_text,
        color='black',
        fillColor=color,
        fillOpacity=0.8,
        weight=2
    ).add_to(m)

# Save map
m.save('plots/aqi_spatial_map.html')
print("✅ Interactive AQI map saved to plots/aqi_spatial_map.html")

## 7. Feature Engineering Insights

In [ ]:
# Create lag features for time series analysis
print("=== FEATURE ENGINEERING ANALYSIS ===")

# Create sample lag features for one city
delhi_data = df[df['city'] == 'Delhi'].copy().sort_values('timestamp')
delhi_data['aqi_lag1'] = delhi_data['aqi'].shift(1)
delhi_data['aqi_lag6'] = delhi_data['aqi'].shift(6)
delhi_data['aqi_lag24'] = delhi_data['aqi'].shift(24)
delhi_data['aqi_rolling_6h'] = delhi_data['aqi'].rolling(window=6).mean()
delhi_data['aqi_rolling_24h'] = delhi_data['aqi'].rolling(window=24).mean()

# Calculate correlations with current AQI
lag_features = ['aqi_lag1', 'aqi_lag6', 'aqi_lag24', 'aqi_rolling_6h', 'aqi_rolling_24h']
lag_correlations = delhi_data[['aqi'] + lag_features].corr()['aqi'].drop('aqi')

print("\nCorrelations between current AQI and lag features:")
print(lag_correlations.sort_values(ascending=False))

# Feature importance simulation (using random forest)
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# Prepare features for importance analysis
feature_df = df.copy()
le = LabelEncoder()
feature_df['city_encoded'] = le.fit_transform(feature_df['city'])
feature_df['season_encoded'] = le.fit_transform(feature_df['season'])

feature_columns = ['pm25', 'pm10', 'no2', 'o3', 'so2', 'co', 'temperature', 
                  'humidity', 'wind_speed', 'pressure', 'month', 'hour', 
                  'day_of_week', 'city_encoded', 'season_encoded']

X = feature_df[feature_columns]
y = feature_df['aqi']

# Fit random forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X, y)

# Get feature importance
feature_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== FEATURE IMPORTANCE (Random Forest) ===")
print(feature_importance)

In [ ]:
# Visualize feature importance
plt.figure(figsize=(12, 8))
sns.barplot(data=feature_importance.head(10), x='importance', y='feature', palette='viridis')
plt.title('Top 10 Most Important Features for AQI Prediction', fontsize=16, fontweight='bold')
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.tight_layout()
plt.savefig('plots/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Feature importance plot saved to plots/feature_importance.png")

## 8. Summary and Key Insights

In [ ]:
print("\n" + "="*60)
print("              KEY INSIGHTS FROM EDA")
print("="*60)

print("\n🔍 DATA QUALITY:")
print(f"   • Dataset contains {len(df):,} records across {df['city'].nunique()} cities")
print(f"   • Time period: {df['timestamp'].min().strftime('%Y-%m-%d')} to {df['timestamp'].max().strftime('%Y-%m-%d')}")
print(f"   • No missing values detected")
print(f"   • AQI range: {df['aqi'].min():.1f} to {df['aqi'].max():.1f}")

print("\n📊 POLLUTION PATTERNS:")
most_polluted = df.groupby('city')['aqi'].mean().idxmax()
least_polluted = df.groupby('city')['aqi'].mean().idxmin()
print(f"   • Most polluted city: {most_polluted} (avg AQI: {df.groupby('city')['aqi'].mean().max():.1f})")
print(f"   • Least polluted city: {least_polluted} (avg AQI: {df.groupby('city')['aqi'].mean().min():.1f})")

worst_month = df.groupby('month')['aqi'].mean().idxmax()
best_month = df.groupby('month')['aqi'].mean().idxmin()
print(f"   • Worst pollution month: {worst_month} (avg AQI: {df.groupby('month')['aqi'].mean().max():.1f})")
print(f"   • Best pollution month: {best_month} (avg AQI: {df.groupby('month')['aqi'].mean().min():.1f})")

worst_hour = df.groupby('hour')['aqi'].mean().idxmax()
best_hour = df.groupby('hour')['aqi'].mean().idxmin()
print(f"   • Peak pollution hour: {worst_hour}:00 (avg AQI: {df.groupby('hour')['aqi'].mean().max():.1f})")
print(f"   • Lowest pollution hour: {best_hour}:00 (avg AQI: {df.groupby('hour')['aqi'].mean().min():.1f})")

print("\n🔗 CORRELATIONS:")
aqi_corr = df[['aqi', 'pm25', 'pm10', 'temperature', 'humidity', 'wind_speed']].corr()['aqi']
strongest_positive = aqi_corr.drop('aqi').max()
strongest_negative = aqi_corr.drop('aqi').min()
print(f"   • Strongest positive correlation: {aqi_corr.drop('aqi').idxmax()} (r={strongest_positive:.3f})")
print(f"   • Strongest negative correlation: {aqi_corr.drop('aqi').idxmin()} (r={strongest_negative:.3f})")

print("\n🏆 TOP PREDICTIVE FEATURES:")
top_features = feature_importance.head(5)
for i, (_, row) in enumerate(top_features.iterrows(), 1):
    print(f"   {i}. {row['feature']}: {row['importance']:.3f}")

print("\n📈 SEASONAL INSIGHTS:")
seasonal_stats = df.groupby('season')['aqi'].agg(['mean', 'std']).round(1)
for season, stats in seasonal_stats.iterrows():
    print(f"   • {season}: {stats['mean']} ± {stats['std']} AQI")

print("\n🎯 MODELING RECOMMENDATIONS:")
print("   • Use PM2.5 and PM10 as primary features (strong correlation with AQI)")
print("   • Include temporal features (hour, month, season) for capturing patterns")
print("   • Consider lag features (previous hours) for time series modeling")
print("   • Weather features (wind speed, temperature) are important predictors")
print("   • City-specific models may perform better due to varying base pollution levels")
print("   • Focus on winter months and rush hours for critical pollution events")

print("\n" + "="*60)

In [ ]:
# Save summary statistics to file
summary_stats = {
    'dataset_info': {
        'total_records': len(df),
        'cities': list(df['city'].unique()),
        'date_range': f"{df['timestamp'].min()} to {df['timestamp'].max()}",
        'aqi_range': f"{df['aqi'].min():.1f} to {df['aqi'].max():.1f}"
    },
    'city_rankings': {
        'most_polluted': df.groupby('city')['aqi'].mean().sort_values(ascending=False).to_dict(),
        'pollution_variability': df.groupby('city')['aqi'].std().sort_values(ascending=False).to_dict()
    },
    'temporal_patterns': {
        'monthly_avg': df.groupby('month')['aqi'].mean().to_dict(),
        'hourly_avg': df.groupby('hour')['aqi'].mean().to_dict(),
        'seasonal_avg': df.groupby('season')['aqi'].mean().to_dict()
    },
    'correlations': df[['aqi', 'pm25', 'pm10', 'temperature', 'humidity', 'wind_speed']].corr()['aqi'].to_dict(),
    'feature_importance': feature_importance.set_index('feature')['importance'].to_dict()
}

import json
with open('eda_summary.json', 'w') as f:
    json.dump(summary_stats, f, indent=2, default=str)

print("✅ EDA summary saved to eda_summary.json")
print("✅ All plots saved to plots/ directory")
print("\n🎉 Exploratory Data Analysis Complete!")